<a href="https://colab.research.google.com/github/t1nh233/predict_vnindex_stacked_lstm/blob/main/lstm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import numpy as np
import torch

In [29]:
## LOAD VA CLEAN DU LIEU DAU VAO

# Doc du lieu
url = "https://raw.githubusercontent.com/t1nh233/predict_vnindex_stacked_lstm/refs/heads/main/vn_index_historical_data_9_12.csv"
vnindex = pd.read_csv(url)
vnindex.head()

# # Xu ly du lieu thoi gian va dua Date thanh index
vnindex['Date'] = pd.to_datetime(vnindex['Date'], format='%m/%d/%Y')
vnindex = vnindex.sort_values(by='Date')
vnindex.set_index('Date', inplace=True)

# Chuyen doi ve dung dinh dang va dung don vi
columns_convert_to_float = ["Price", "Open", "High", "Low"]
for column in columns_convert_to_float:
    vnindex[column] = vnindex[column].str.replace(",", "").astype(float)

# Ham ho tro chuyen doi Volume va Change
def convert_volume(vol_str):
    multiplier_dict = {"K": 1000, "M": 1000000, "B": 1000000000}

    if not isinstance(vol_str, str):
        return vol_str

    vol_str = vol_str.upper().strip()
    last_char = vol_str[-1]

    if last_char in multiplier_dict.keys():
        multiplier_val = multiplier_dict[last_char]
        number_part = vol_str[:-1]
    else:
        multiplier_val = 1
        number_part = vol_str

    try:
        number = float(number_part.replace(',', ''))
    except ValueError:
        return np.nan

    return number * multiplier_val

def convert_change(change_str):
    if not isinstance(change_str, str):
        return change_str

    if change_str.endswith('%'):
        number_part = change_str[:-1]
    else:
        number_part = change_str

    try:
        number = float(number_part.replace(',', ''))
    except ValueError:
        return np.nan

    return number / 100.0


# Bien doi Volume va Change
vnindex['Vol.'] = vnindex['Vol.'].apply(convert_volume)
vnindex['Change %'] = vnindex['Change %'].apply(convert_change)

vnindex.rename(columns={'Price': 'Close', 'Vol.': 'Volume'}, inplace=True)

print(vnindex.info())
print(vnindex.head())

## DONE PHAN LOAD VA CLEAN DU LIEU DAU VAO

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3743 entries, 2010-12-09 to 2025-12-09
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Close     3743 non-null   float64
 1   Open      3743 non-null   float64
 2   High      3743 non-null   float64
 3   Low       3743 non-null   float64
 4   Volume    3743 non-null   float64
 5   Change %  3743 non-null   float64
dtypes: float64(6)
memory usage: 204.7 KB
None
             Close    Open    High     Low    Volume  Change %
Date                                                          
2010-12-09  460.45  450.59  460.79  450.16   60570.0    0.0168
2010-12-10  473.06  464.81  473.06  462.39   94110.0    0.0274
2010-12-13  490.22  485.82  490.30  485.73   81980.0    0.0363
2010-12-14  489.65  493.72  496.42  480.64  134950.0   -0.0012
2010-12-15  493.47  492.80  497.09  491.51   95680.0    0.0078


In [20]:
## CHUAN BI DU LIEU CHO MO HINH